In [3]:
import numpy as np
import scipy.optimize
import matplotlib.pyplot as plt
from scipy.linalg import expm

# Construction de la matrice A de vectorisation

In [35]:
def Hamiltonian(omega : float, u : float, kappa : float ) :
    H = np.array([[omega, u*kappa],[u*kappa,-omega]])
    return H 



# definir les constantes du probleme : 
u = 1 # controle 
omega = 5
kappa = 8
gamma_1 = 3
gamma_2 = 4

# les matrices de Pauli 

X = np.array([[0, 1],[1,0]])
Y = np.array([[0, complex(0,-1)],[complex(0,1),0]])
Z = np.array([[1, 0],[0,-1]])

#Les matrices Sigma : 
Sigma_plus = X + complex(0,1) * Y
Sigma_minus = X - complex(0,1) * Y

H = Hamiltonian(omega,u,kappa)
I = np.eye(2)

# La matrice de vectorisation A : 
A = np.kron(I,H) - np.kron(H,I) + gamma_1*(np.kron(Sigma_minus.conj().T,Sigma_plus) - 0.5*(np.kron(I,np.dot(Sigma_minus,Sigma_plus)) + np.kron(np.matmul(Sigma_minus,Sigma_plus).conj().T,I) )) + gamma_2*(np.kron(Z,Z))


# Résolution de l'equation de Linblad
def V_cal(t : float, A : np.array, V0) -> np.array : 
    return np.matmul(np.exp(t*A),V0)


# Dans notre cas : Si V0 = vec(|0><0|)
V0 = np.array([1,0,0,0])
V = V_cal(2,A,V0)
print(V)

[2.98095799e+03+0.j 8.88611052e+06+0.j 1.12535175e-07+0.j
 1.00000000e+00+0.j]


# Collection de la Data à plusieurs temps de mesure : 

On se donne un operateur de mesure E0 = |0> <0|, la probability d'observer l'etat pur E0 est donner par $tr(E_1AE_1)$
On la modelise par une experience de Bernouilli de paramétre $p = tr(E_1AE_1)$



In [45]:
def Observation(V,t:float,nb_experience : int) : 
    V = V_cal(t,A,V0)
    rho = np.array([[V_t[0], V_t[2]], 
                    [V_t[1], V_t[3]]], dtype=np.complex128)
    E0 = np.array([[1, 0], [0, 0]])
    p = np.trace(E0 @ rho @ E0).real
    return np.random.binomial(n=1, p=p, size=nb_experience)

In [46]:
    # Calcul de V(t) pour t = 2
    V0 = np.array([1, 0, 0, 0], dtype=np.complex128)
    t = 2
    V_t = V_cal(t, A, V0)
    print(f"V(2) = {V_t}")

    # Simulation de 1000 mesures
    nb_experience = 1000
    observations = Observation(V, t, nb_experience)

V(2) = [2.98095799e+03+0.j 8.88611052e+06+0.j 1.12535175e-07+0.j
 1.00000000e+00+0.j]


ValueError: p < 0, p > 1 or p is NaN

2980.9579870417283